In [1]:
import os 

In [2]:
%pwd

'e:\\SummerProgram\\Projects\\Image_Bases_DL_Projects\\End_To_End_Chicken_Disease_Classifier\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'e:\\SummerProgram\\Projects\\Image_Bases_DL_Projects\\End_To_End_Chicken_Disease_Classifier'

In [ ]:
# 1- config/config.yaml ----- for evaluation we don't need the config 
# 2. src/cnnClassifier/entity/config_entity
# 3. src/cnnClassifier/config/configuration.py

In [5]:
#2. entity 
from dataclasses import dataclass
from pathlib import Path 

@dataclass(frozen = True)
class EvaluationConfig:
    path_of_model : Path 
    training_data : Path 
    all_params : dict
    params_image_size : list 
    params_batch_size : int
    
    
    


In [6]:
# 3 Configuration Maneger 
from cnnClassifier.constants import * 
from cnnClassifier.utils.common import read_yaml , create_directories , save_json



In [7]:
class ConfiguartionManeger:
    def __init__(self , config_filepath = CONFIG_FILE_PATH , params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
        
        
    def get_validation_config(self)-> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model= "artifacts/training/model.keras",
            training_data= "artifacts/data_ingestion/Chicken-fecal-images",
            all_params = self.params,
            params_image_size= self.params.IMAGE_SIZE ,
            params_batch_size= self.params.BATCH_SIZE
            
        )
        return eval_config
        
        
        
        

In [8]:
# 3 . Component - Model Evaluation Stage
from urllib.parse import urlparse
import tensorflow as tf 

In [9]:
model = tf.keras.models.load_model("artifacts/training/model.keras")


In [12]:
class Evaluation:
    def __init__(self , config : EvaluationConfig):
        self.config = config
        
    def _valid_generator(self):
        datagenerator_kwargs =dict(
            rescale = 1.0/255,
            validation_split = 0.20   
        )
        
        
        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = "bilinear"
        )
        
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )
        
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory = self.config.training_data,
            subset = "validation",
            shuffle = False,
            **dataflow_kwargs
        )
        
        
    @staticmethod
    def load_model(path:Path)->tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    
    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        
    def save_score(self):
        scores = {"loss": self.score[0] , "accuracy" : self.score[1]}
        save_json(path=Path("score.json") , data=scores)
        
        

In [13]:
# 5 Create the Pipeline

try:
    config = ConfiguartionManeger()
    val_config = config.get_validation_config()
    evaluation = Evaluation(config = val_config)
    evaluation.evaluation()
    evaluation.save_score()
    
except Exception as e:
    raise e

[2026-02-11 00:55:29,510: INFO: common: yaml file: config\config.yaml loaded successfully]
[2026-02-11 00:55:29,519: INFO: common: yaml file: params.yaml loaded successfully]
[2026-02-11 00:55:29,525: INFO: common: created directory at: artifacts]
Found 78 images belonging to 2 classes.
3/3 ━━━━━━━━━━━━━━━━━━━━ 6s 2s/step - accuracy: 0.5000 - loss: 1.1145  
[2026-02-11 00:55:35,859: INFO: common: json file saved at: score.json]
